### Objective:
To classify job descriptions into standardized ESCO occupation codes using Sentence-BERT embeddings and Linear Support Vector Machine (SVM).

### Importing Required Libraries

In [7]:
import pandas as pd
import numpy as np
import re
from collections import Counter #To identify most common job codes.
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC #Classifier for multi-class SVM.
from sentence_transformers import SentenceTransformer
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder #Convert ESCO codes to numerical labels.
from sentence_transformers import SentenceTransformer #Generate dense semantic embeddings of job descriptions using pretrained BERT-like models.

### Load the ESCO Occupation Dataset

In [8]:
df = pd.read_csv('C:/Users/hp/Desktop/Machine Learning/Datasets/ESCO dataset - v1.2.0 - classification - en - csv/occupations_en.csv')

### Data Preprocessing and Text Cleaning

In [9]:
def clean_text(text): #removes punctuation, trims spaces, and lowercases the text.
    """Clean job description text."""
    text = str(text).lower().strip()
    text = re.sub(r'\s+', ' ', text)  # Remove extra spaces
    text = re.sub(r'[^\w\s]', '', text)  # Remove punctuation
    return text

# Combine multiple text columns to create a richer description
df['job_text'] = (
    df['preferredLabel'].fillna('').map(clean_text) + ' ' +
    df['altLabels'].fillna('').map(clean_text) + ' ' +
    df['definition'].fillna('').map(clean_text) + ' ' +
    df['description'].fillna('').map(clean_text)
)

### Filter Dataset to Top-N Frequent Classes (Reducing Noise)

In [10]:
#Focuses on the top 500 most frequent job codes to reduce class imbalance and improve model performance.
top_n = 500
top_classes = [label for label, _ in Counter(df['code']).most_common(top_n)]

# Filter the DataFrame to keep only the top N frequent job classes
mask = np.isin(df['code'], top_classes)
df_filtered = df[mask]

### Feature Engineering - Convert Job Text to Embeddings
**Sentence Embeddings with BERT**

In [11]:
# Initialize the BERT model (Sentence-BERT)
model_name = 'all-MiniLM-L6-v2'  # Pre-trained model for sentence embeddings
embedder = SentenceTransformer(model_name)

# Convert job descriptions to BERT embeddings (vector representations)
X = embedder.encode(df_filtered['job_text'].tolist(), show_progress_bar=True)

C:\Users\hp\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Batches:   0%|          | 0/16 [00:00<?, ?it/s]

### Encode Target Labels (ESCO Codes)

In [12]:
#Converts job codes (strings like '2144.1.8') into numerical labels for training.
#Later used to decode predictions back into ESCO codes.

le = LabelEncoder()
y = le.fit_transform(df_filtered['code'])

### Split Data into Train and Test Sets

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

### Train the Model with Class Weighting to Handle Class Imbalance

In [14]:
model = LinearSVC(class_weight='balanced') # Automatically balance class weights.(LVC = multi-class classification)
model.fit(X_train, y_train)

LinearSVC(class_weight='balanced')

### Predict and Evaluate the Model

In [15]:
y_pred = model.predict(X_test)

# Get unique labels in y_test and y_pred
unique_labels_in_pred = np.unique(y_pred)
unique_labels_in_test = np.unique(y_test)

# Combine the unique labels from both y_test and y_pred to cover all classes
all_unique_labels = np.union1d(unique_labels_in_pred, unique_labels_in_test)

# Get the class names corresponding to those labels
# This ensures target_names corresponds to the labels that the model actually predicted
target_names = le.inverse_transform(all_unique_labels)

# Now print the classification report
print("\nClassification Report:")
print(classification_report(y_test, y_pred, target_names=target_names))


Classification Report:
               precision    recall  f1-score   support

      0110.11       0.00      0.00      0.00       1.0
       0110.9       0.00      0.00      0.00       0.0
       1111.6       0.00      0.00      0.00       1.0
       1112.3       0.00      0.00      0.00       0.0
     1112.3.1       0.00      0.00      0.00       1.0
       1112.6       0.00      0.00      0.00       0.0
     1211.1.1       0.00      0.00      0.00       1.0
       1219.2       0.00      0.00      0.00       0.0
       1221.1       0.00      0.00      0.00       1.0
       1221.3       0.00      0.00      0.00       1.0
 1221.3.2.1.3       0.00      0.00      0.00       1.0
     1223.2.1       0.00      0.00      0.00       1.0
   1223.2.1.1       0.00      0.00      0.00       0.0
   1321.2.2.1       0.00      0.00      0.00       1.0
   1321.2.2.2       0.00      0.00      0.00       0.0
1324.3.1.6.11       0.00      0.00      0.00       0.0
1324.3.1.6.13       0.00      0.00      

C:\Users\hp\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\hp\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\hp\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetric

# Predict and Display the Results

In [16]:
from IPython.display import Markdown, display

def predict_and_display_poster(text):
    # Generate embedding
    features = embedder.encode([text])
    predicted_label = model.predict(features)[0]
    job_uri = le.inverse_transform([predicted_label])[0]
    match = df[df['code'] == job_uri]

    if not match.empty:
        job_title = match['preferredLabel'].values[0].title()
        definition = match['definition'].values[0] if pd.notna(match['definition'].values[0]) else "No detailed definition available."
    else:
        job_title = "Unknown"
        definition = "No information found for this prediction."

    poster = f"""
# 🧠 Job Role Classifier
---
### 📝 Input Job Description
> *{text.capitalize()}*
---
### 🎯 Predicted Role
**{job_title}**
---
### 🔗 ESCO Code
`{job_uri}`
---
### 📖 What Does This Role Do?
{definition}
---
🔄 *Prediction generated using AI-powered ESCO classifier.*

    """
    display(Markdown(poster))

In [17]:
predict_and_display_poster("installs and maintains industrial cooling systems")


# 🧠 Job Role Classifier
---
### 📝 Input Job Description
> *Installs and maintains industrial cooling systems*
---
### 🎯 Predicted Role
**Heating, Ventilation, Air Conditioning Engineer**
---
### 🔗 ESCO Code
`2144.1.8`
---
### 📖 What Does This Role Do?
No detailed definition available.
---
🔄 *Prediction generated using AI-powered ESCO classifier.*

    